**CSI 5139 Assignment 3**

This assignment explores transfer learning.

# Setup

In [38]:
import os
from shutil import copy

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model, Model

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

# VGG-16

In [80]:
#datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator(rescale=1./255)

train_batchsize = 32 # 160 images per 68 classes
test_batchsize = 4 # 20 images per 68 classes

train_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/train/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=train_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/test/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=test_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

print("Train generator length: ", len(train_generator))
print("Test generator length: ", len(test_generator))
print("Target size: ", train_generator.target_size)
#print("Generator indices: ", train_generator.class_indices)
#print("Generator filenames:", train_generator.filenames[:10])

Found 10880 images belonging to 68 classes.
Found 1360 images belonging to 68 classes.
Train generator length:  340
Test generator length:  340
Target size:  (128, 128)


#### Model

In [87]:
def vgg16(freeze=False, remove=False):
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
       
    if remove:
        # Remove the layers
        layers = vgg_conv.layers
        for i in range(8):
            layers.pop()
    
    if freeze:
        # Freeze the layers
        # train the new layers with the weights of the existing VGG layers ﬁxe
        for layer in vgg_conv.layers:
            layer.trainable = False
    
    # Check individual layers
    for layer in vgg_conv.layers:
        print(layer, layer.trainable)
        
    return vgg_conv

#### Run model

In [89]:
epochs = 1

vgg_conv = vgg16(freeze=True, remove=True)
vgg_conv.summary()

model = Sequential()
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(68, activation = 'softmax'))

model.layers[0].trainable = False

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

model.summary()

rmsprop = RMSprop()
#sgd = SGD(lr=1e-3)
#sgd = SGD()

checkpoint_path = "./runs/vgg16.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model.compile(optimizer = rmsprop, loss = 'categorical_crossentropy', metrics = ['accuracy'])

hist_vgg16 = model.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1,
        callbacks = [cp_callback])

# save history
hist_vgg16 = hist_vgg16.history
with open('./runs/vgg16.hist', 'w', encoding="utf-8") as fout:
    pprint(hist_vgg16, fout)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000002AE15B79AC8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE15B79550> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE043E59E8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002AE15B79898> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE19BBE438> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE19B84CF8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002AE55ACC278> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE23E8BA58> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE23E9A320> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002AE02FFEA20> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002AF1B6A5C88> False
___________________

KeyError: 'block5_pool_ib-0'

# MNIST VGG-16

In [83]:
vgg_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))

print(vgg_model.summary())

layers = vgg_model.layers
for i in range(8):
    layers.pop()
    
for layer in vgg_model.layers:
    layer.trainable = False

# Check individual layers
for layer in vgg_model.layers:
    print(layer, layer.trainable)
        
print(vgg_model.summary())

# last layers output
x = vgg_model.layers[-1].output
# Flatten as before
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(68, activation='softmax')(x)

vgg_model_transfer = Model(inputs=vgg_model.input, outputs=x)

print(vgg_model_transfer.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [84]:
epochs = 1

#sgd = SGD(lr=0.0001)
#sgd = SGD()
rmsprop = RMSprop(lr=0.0008)
vgg_model_transfer.compile(loss='categorical_crossentropy', 
                           optimizer=rmsprop, 
                           metrics=['accuracy'])

hist_vgg16_model = vgg_model_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1,)

Epoch 1/1
 14/340 [>.............................] - ETA: 23:40 - loss: 15.5900 - acc: 0.0246

KeyboardInterrupt: 